# Library

In [ ]:
library(MASS)
library(lmtest)
library(car)
library(tseries)
library(tidyverse)
library(readxl)
library(plm)
library(ggplot2)

# Data

In [ ]:
path = "/kaggle/input/data-semifinal-sac-ipb-23/Data Semifinal.xlsx"
data = read_excel(path)
data

# Preprocessing & EDA

> ## Ubah Nama Kolom

In [ ]:
colnames(data)

In [ ]:
# Ubah Nama Kolom
nama_kol = c("No", "Kode", "Daerah", "Tahun", 'Latitude', 'Longitude', 'Stunting', 'Sanitasi', 'PHBS', 'AHH', 'PDRB', 'Miskin', 
             'Pengeluaran', 'Angkatan_Kerja', 'Penduduk_Bekerja', 'TPAK', 'TPT', 'RLS')
names(data) <- nama_kol
head(data)

In [ ]:
keluar_kol = c("No", "Kode", "Latitude", "Longitude", "Sanitasi", "PHBS")
data1 = data[, !(colnames(data) %in% keluar_kol)]
head(data1)

> ## Pengecekan & Imputasi Missing Value<br>
Data Stunting 2022

In [ ]:
missing.values <- data1[3:12] %>%
    gather(key = "key", value = "val") %>%
    mutate(is.missing = is.na(val)) %>%
    group_by(key, is.missing) %>%
    summarise(num.missing = n()) %>%
    filter(is.missing==T) %>%
    select(-is.missing) %>%
    arrange(desc(num.missing)) 

missing.values["Persen"] = missing.values$num.missing/nrow(data1)
missing.values

In [ ]:
# unique(data$Daerah)

In [ ]:
# Imputasi Data Stunting 2022
stunt22 = c(24.9, 27.5, 13.6, 25, 23.6, 27.2, 18.6, 19.4, 18.6, 24.3,
           27.6, 21.1, 15.7, 21.8, 14, 17.8, 27.3, 20, 24.9, 19.2,
           19.4, 17, 6, 12.6, 16.4, 22.4, 19.3)
data1$Stunting[1:27] = stunt22
head(data1, 27)

> ## Daerah, Tahun X TPT

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 12)

ggplot(data1,aes(y= TPT,x=reorder(Daerah, Tahun),fill=factor(Tahun)))+
  geom_bar(position = 'dodge',stat='identity')+
  labs(x='Kabupateb/Kota',y='TPT',fill='Tahun')+
  theme(axis.text.x = element_text(angle=90, vjust=0.5, hjust = 1))

ggsave("output/eda_hist.png")

> ## Daerah x TPT

In [ ]:
# install.packages("gplots")

In [ ]:
library(gplots)
plotmeans(TPT~Daerah , main = "Daerah X TPT", data = data1, las=2)

ggsave("output/daerah x tpt.png")

> ## Tahun X TPT

In [ ]:
plotmeans(TPT~Tahun , main = "Tahun X TPT", data = data1)
ggsave("output/eda_tahun x tpt.png")

> ## Sebaran Data Tiap Variabel

In [ ]:
data_long <- gather(data1[3:12], key = "Variabel", value = "Nilai")
head(data_long)

ggplot(data_long, aes(x = Variabel, y = Nilai, fill = Variabel)) +
  geom_boxplot() +
  labs(x = "Variabel", y = "Nilai") +
  theme_classic()

ggsave("output/eda_boxp.png")

> ## Imbalence Data

In [ ]:
data1 %>% group_by(Daerah) %>% summarize(length(Daerah))

> ## Scaling Data

In [ ]:
str(data1)

In [ ]:
summary(data1)

In [ ]:
num_col = c(3:12)
data2 = data1
data2[num_col] = lapply(data2[num_col], scale)
head(data2, 10)

In [ ]:
summary(data2)

# PANEL MODEL & ANALISIS

> ## Baseline

In [ ]:
# Tanpa Daerah & Tahun
data3 = data2[-c(1, 2)]
head(data3)

In [ ]:
#Model 1
lm1 = lm(TPT~., data=data3)
summary(lm1)

In [ ]:
#Model 2
lm2 = lm(TPT~.-Pengeluaran, data=data3)
summary(lm2)

In [ ]:
#Model 3
lm3 = lm(TPT~.-Pengeluaran-Stunting, data=data3)
summary(lm3)

In [ ]:
#Model 4
lm4 = lm(TPT~.-Pengeluaran-Stunting-RLS, data=data3)
summary(lm4)

> # Haussman Test<br>
H0: Model REM yang sesuai<br>
H1: Model FEM yang sesuai

In [ ]:
names(lm1$coefficients)[-1]

In [ ]:
gf1 = plm(TPT~Stunting+AHH+PDRB+Miskin+Pengeluaran+Angkatan_Kerja+Penduduk_Bekerja+TPAK+RLS, data = data2, model = "within", index=c("Daerah","Tahun"))
gr1 = plm(TPT~Stunting+AHH+PDRB+Miskin+Pengeluaran+Angkatan_Kerja+Penduduk_Bekerja+TPAK+RLS, data = data2, model = "random", index=c("Daerah","Tahun"))
phtest(gf1, gr1)

In [ ]:
gf2 = plm(TPT~Stunting+AHH+PDRB+Miskin+Angkatan_Kerja+Penduduk_Bekerja+TPAK+RLS, data = data2, model = "within", index=c("Daerah","Tahun"))
gr2 = plm(TPT~Stunting+AHH+PDRB+Miskin+Angkatan_Kerja+Penduduk_Bekerja+TPAK+RLS, data = data2, model = "random", index=c("Daerah","Tahun"))
phtest(gf2, gr2)

In [ ]:
gf3 = plm(TPT~AHH+PDRB+Miskin+Angkatan_Kerja+Penduduk_Bekerja+TPAK+RLS, data = data2, model = "within", index=c("Daerah","Tahun"))
gr3 = plm(TPT~AHH+PDRB+Miskin+Angkatan_Kerja+Penduduk_Bekerja+TPAK+RLS, data = data2, model = "random", index=c("Daerah","Tahun"))
phtest(gf3, gr3)

In [ ]:
gf4 = plm(TPT~AHH+PDRB+Miskin+Angkatan_Kerja+Penduduk_Bekerja+TPAK, data = data2, model = "within", index=c("Daerah","Tahun"))
gr4 = plm(TPT~AHH+PDRB+Miskin+Angkatan_Kerja+Penduduk_Bekerja+TPAK, data = data2, model = "random", index=c("Daerah","Tahun"))
phtest(gf4, gr4)

> # Chow Test<br>
H0: Model CEM yang sesuai<br>
H1: Model FEM yang sesuai

In [ ]:
gc1 = plm(TPT~Stunting+AHH+PDRB+Miskin+Pengeluaran+Angkatan_Kerja+Penduduk_Bekerja+TPAK+RLS, data = data2, model = "pooling", index=c("Daerah","Tahun"))
pFtest(gf1, gc1)

In [ ]:
gc2 = plm(TPT~Stunting+AHH+PDRB+Miskin+Angkatan_Kerja+Penduduk_Bekerja+TPAK+RLS, data = data2, model = "pooling", index=c("Daerah","Tahun"))
pFtest(gf2, gc2)

In [ ]:
gc3 = plm(TPT~AHH+PDRB+Miskin+Angkatan_Kerja+Penduduk_Bekerja+TPAK+RLS, data = data2, model = "pooling", index=c("Daerah","Tahun"))
pFtest(gf3, gc3)

In [ ]:
gc4 = plm(TPT~AHH+PDRB+Miskin+Angkatan_Kerja+Penduduk_Bekerja+TPAK, data = data2, model = "pooling", index=c("Daerah","Tahun"))
pFtest(gf4, gc4)

> # Diagnostic

In [ ]:
pbgtest(gf1,order=2)

In [ ]:
pbgtest(gf2,order=2)

In [ ]:
pbgtest(gf3,order=2)

In [ ]:
pbgtest(gf4,order=2)

In [ ]:
bptest(gf1)

In [ ]:
bptest(gf2)

In [ ]:
bptest(gf3)

In [ ]:
bptest(gf4)

> # Model Terbaik

In [ ]:
summary(gf1)
summary(gf2)
summary(gf3)
summary(gf4)

In [ ]:
summary(gf2)

In [ ]:
sort(fixef(gf2))

> ## Penanganan

In [ ]:
# Membuat model dengan data yang sudah discaling dan ditransformasi
# model_lag_log <- plm(log(TPT) ~ lag(log(Stunting)) + lag(log(AHH)) + lag(log(PDRB)) + log(Miskin) + log(Angkatan_Kerja) + log(Penduduk_Bekerja) + log(TPAK) + log(RLS), data = data, model = "within", index = c("Daerah", "Tahun"))
gf2_log <- plm(log(TPT)~Stunting+AHH+PDRB+Miskin+Angkatan_Kerja+Penduduk_Bekerja+TPAK+RLS, data = data2, model = "within", index=c("Daerah","Tahun"))
pbgtest(gf2_log, order = 2)
bptest(gf2_log)

In [ ]:
summary(gf2_log)

In [ ]:
sort(fixef(gf2))